<a href="https://colab.research.google.com/github/youznn/LLM-PJT/blob/main/RAG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets llama-index==0.10.34 langchain-openai==0.1.6 "nemoguardrails[openai]==0.8.0" openai==1.25.1 chromadb==0.5.0 wandb==0.16.6 -qqq
!pip install llama-index-callbacks-wandb==0.1.2 -qqq

허깅페이스, 라마인덱스, 랭체인, openAI 패키지 설치

In [4]:
import os
from datasets import load_dataset

os.environ["OPENAI_API_KEY"] = ""
dataset = load_dataset('klue', 'mrc', split='train')
dataset[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

이 데이터셋은 질문과, 그 context를 포함한 쌍을 가진 데이터셋이다.

In [5]:
!pip install openai==0.27.0
!pip install llama-index==0.10.34

  Using cached openai-0.27.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.27.0-py3-none-any.whl (70 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.25.1
    Uninstalling openai-1.25.1:
      Successfully uninstalled openai-1.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-legacy 0.9.48.post4 requires openai>=1.1.0, but you have openai 0.27.0 which is incompatible.
langchain-openai 0.1.6 requires openai<2.0.0,>=1.24.0, but you have openai 0.27.0 which is incompatible.
llama-index-agent-openai 0.2.9 requires openai>=1.14.0, but you have openai 0.27.0 which is incompatible.
  Using cached openai-1.61.0-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.61.0-py3-none-any.whl (460 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.27.0
    Uninstalling openai-0.27.0:
      Succes

In [6]:
from llama_index.core import Document, VectorStoreIndex

text_list = dataset[:100]['context']
documents = [Document(text=t[i:i+500]) for t in text_list for i in range(0, len(t), 500)]


# 인덱스 만들기
index = VectorStoreIndex.from_documents(documents)

처음 100개의 context를 뽑아서 text_list로 만들고 Document 객체로 변환한다. 이후, 인덱스를 생성한다.

In [13]:
print(dataset[0]['question']) # 북태형양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?


retrieval_engine = index.as_retriever(similarity_top_k=5, verbose=True)
response = retrieval_engine.retrieve(
    dataset[0]['question']
)

print(len(response))
print(response[0].node.text)

북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
5
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳


`similarity top` -> 입력 질문과 가장 유사한 5개의 기사 본문을 찾아 반환함

In [19]:
# 라마인덱스를 활용해 검색 증강 생성 수행

# index를 쿼리 엔진으로 변환함
query_engine = index.as_query_engine(similarity_top_k=1)

# 기사 본문을 프롬프트에 추가 + LLM 답변까지 생성
response = query_engine.query(
    dataset[0]['question']
)

print(response)

한 달가량


## 내부적으로는 어떻게 수행될까?
1. `VectorIndexRetriever` 클래스를 사용해 벡터 데이터베이스에서 검색하는 `retriever` 만든다.
2. `get_response_synthesizer()` 함수를 사용해 프롬프트를 통합할 때 사용할 `response_synthesizer`를 만든다.
3. `RetrieverQueryEngine` 클래스에 앞서 생성한 `retriver`와 `response_syntehsizer`를 전달해 검색 증강 생성을 한번에 수행하는 `query_engine`을 생성한다. 이때 `SimilarityPostprocessor`와 같은 클래스를 사용해 질문과 유사도가 낮은 경우는 필터링하도록 설정



In [22]:
# LLM 캐시 사용
import os
import chromadb
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = ""
openai_client = OpenAI()
chroma_client = chromadb.Client()

In [24]:
##LLM 캐시를 사용하지 않았을 때

import time

def response_text(openai_resp):
    return openai_resp.choices[0].message.content

question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
for _ in range(2):
    start_time = time.time()
    response = openai_client.chat.completions.create(
      model='gpt-3.5-turbo',
      messages=[
        {
            'role': 'user',
            'content': question
        }
      ],
    )
    response = response_text(response)
    print(f'질문: {question}')
    print("소요 시간: {:.2f}s".format(time.time() - start_time))
    print(f'답변: {response}\n')

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 1.85s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울까지이며, 10월부터 3월 사이에 국내를 지나가는 것이 일반적입니다. 이 기간 동안 한반도 지역은 강추위가 빈번하게 찾아와 추운 겨울철을 맞이하게 됩니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 1.58s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 주로 가을부터 겨울까지이며, 대략 3~4개월 정도의 기간 동안 한반도 지역을 영향하게 됩니다. 이 기간 동안 한반도 지역은 추운 날씨와 강한 바람을 경험하게 되며, 대기 중의 미세먼지 농도도 높아지는 경향이 있습니다.



In [26]:
# 캐시 구현해 보자!!
class OpenAICache:
    def __init__(self, openai_client):
        self.openai_client = openai_client
        self.cache = {}

    def generate(self, prompt):
        if prompt not in self.cache:
            response = self.openai_client.chat.completions.create(
                model = 'gpt-3.5-turbo',
                messages = [
                    {
                        'role': 'user',
                        'content': prompt
                    }
                ],
            )
            self.cache[prompt] = response_text(response)
        return self.cache[prompt]

openai_cache = OpenAICache(openai_client)

question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
for _ in range(2):
    start_time = time.time()
    response = openai_cache.generate(question)
    print(f'질문: {question}')
    print("소요 시간: {:.2f}s".format(time.time() - start_time))
    print(f'답변: {response}\n')

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 2.41s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 보통 2~3일 정도입니다. 이 기간 동안 한반도 지역에는 고저압이 교차하여 기온 변화가 심하고 강력한 바람과 강수가 동반되는 날씨가 많이 발생합니다. 이 기단의 만남으로 인해 기상 변화가 매우 뚜렷하게 나타나며, 강한 바람과 폭우, 눈 등이 예상될 수 있습니다.

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
소요 시간: 0.00s
답변: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 보통 2~3일 정도입니다. 이 기간 동안 한반도 지역에는 고저압이 교차하여 기온 변화가 심하고 강력한 바람과 강수가 동반되는 날씨가 많이 발생합니다. 이 기단의 만남으로 인해 기상 변화가 매우 뚜렷하게 나타나며, 강한 바람과 폭우, 눈 등이 예상될 수 있습니다.



동일한 질문이 들어왔을 때, 캐시된 데이터를 찾아 반환하기 때문에 소요 시간이 크게 줄어든다.

`sementic_cache` 와 같은 외부 데이터베이스 클라인트 입력으로 받아서, 저장한 다음에 충분히 가까운 거리에 있는 벡터가 있다면 반환한다.